# Exploring OpenAI Image Edit Features via API

This notebook demonstrates how to use the OpenAI API to edit images programmatically. You'll learn how to authenticate, upload images, send edit requests, and visualize results.

## 1. Setup OpenAI API Key

To use the OpenAI API, you'll need an API key. Store your key securely and avoid sharing it publicly.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file.")

import openai
openai.api_key = OPENAI_API_KEY

## 2. Show Image and Create Image Mask

This section allows you to interactively view your image and create a mask using rectangle selection. The mask can be used for image editing with the OpenAI API.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.widgets import RectangleSelector

# --- Load your RGBA image ---
img_path = 'path_to_your_image.png'  # replace with your image path
img = Image.open(img_path).convert('RGBA')
arr = np.array(img)
alpha = arr[:, :, 3]

# --- Generate initial mask of fully transparent pixels ---
initial_mask = (alpha == 0).astype(np.uint8) * 255

# --- Prepare interactive masking ---
mask = np.zeros_like(alpha)  # start with an empty mask

fig, (ax_img, ax_mask) = plt.subplots(1, 2, figsize=(12, 6))
ax_img.imshow(img)
ax_img.set_title('Original Image (Draw boxes here)')
ax_img.axis('off')
ax_mask.imshow(initial_mask, cmap='gray')
ax_mask.set_title('Mask (white = transparent)')
ax_mask.axis('off')

def onselect(eclick, erelease):
    x1, y1 = int(eclick.xdata), int(eclick.ydata)
    x2, y2 = int(erelease.xdata), int(erelease.ydata)
    # Add selected rectangle to mask
    mask[min(y1,y2):max(y1,y2), min(x1,x2):max(x1,x2)] = 255
    # Combine with initial transparency mask if desired:
    combined = np.maximum(initial_mask, mask)
    ax_mask.imshow(combined, cmap='gray')
    fig.canvas.draw()

# Enable rectangle selector on the original image
rect_selector = RectangleSelector(
    ax_img, onselect,
    drawtype='box', useblit=True,
    button=[1],  # use left mouse button
    minspanx=5, minspany=5, 
    spancoords='pixels',
    interactive=True
)

plt.show()

## 3. Upload and Prepare Images

Load an image from your local system and prepare it for editing with the OpenAI API.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Path to your image file
image_path = 'your_image.png'  # Change this to your image file

# Load and display the image
image = Image.open(image_path)
plt.imshow(image)
plt.axis('off')
plt.show()

## 4. Send Image Edit Requests

Use the OpenAI API to send an image edit request. You can specify prompts and parameters for the edit.

In [ ]:
# Example: Send an image edit request (update parameters as needed)
# Note: Replace 'your_image.png' and 'your_mask.png' with your files

response = openai.Image.create_edit(
    image=open('your_image.png', 'rb'),
    mask=open('your_mask.png', 'rb'),  # Optional: provide a mask image
    prompt='A cat wearing sunglasses',
    n=1,
    size='512x512',
    response_format='url',
)

# Extract the URL of the edited image
edited_image_url = response['data'][0]['url']
print('Edited image URL:', edited_image_url)

## 5. Handle API Responses

Parse the response from the API and handle any errors that may occur.

In [ ]:
try:
    edited_image_url = response['data'][0]['url']
    print('Edited image URL:', edited_image_url)
except Exception as e:
    print('Error handling API response:', e)

## 6. Display Edited Images

Visualize the edited image returned by the OpenAI API.

In [ ]:
import requests
from io import BytesIO

# Download and display the edited image
if 'edited_image_url' in locals():
    response_img = requests.get(edited_image_url)
    edited_image = Image.open(BytesIO(response_img.content))
    plt.imshow(edited_image)
    plt.axis('off')
    plt.show()
else:
    print('No edited image URL found.')